#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

#### Pourquoi convertir object en string ?

1. Clarifier le type des données :

    En pandas, une colonne de type object peut contenir divers types de données, comme des chaînes de caractères, des nombres mélangés avec du texte, ou même des valeurs None.
    Pour MongoDB, les valeurs doivent avoir des types clairs et cohérents, car le format BSON exige des types bien définis (string, int, double, boolean, etc.).

2. Faciliter l’insertion dans MongoDB :

    MongoDB traite correctement les chaînes de caractères (string), mais une colonne de type object pourrait provoquer des erreurs si elle contient des types mixtes ou incompatibles.

3. Éviter des comportements inattendus :

    Si des valeurs non compatibles sont insérées dans MongoDB (par exemple, une valeur numérique ou un objet complexe non attendu dans une colonne de texte), cela pourrait entraîner des incohérences dans vos données.

4. Nettoyage des données :

    Cette conversion est également une étape de nettoyage des données, qui vous permet de vérifier si les valeurs de vos colonnes sont bien ce que vous attendez avant de les insérer.

#### Pourquoi convertir en dictionnaire ?

Dans MongoDB, les données sont stockées sous forme de documents BSON (Binary JSON). Lorsque vous utilisez un DataFrame pandas, les données sont structurées en lignes et colonnes, ce qui n'est pas directement compatible avec MongoDB.

La conversion en dictionnaire est nécessaire car :

    Format attendu par pymongo :
        pymongo attend que les données soient sous forme de dictionnaires Python pour les insérer dans MongoDB. Chaque ligne d'un DataFrame devient un dictionnaire, où les clés correspondent aux noms des colonnes et les valeurs aux données.

    Correspondance avec BSON :
        MongoDB stocke les documents sous forme de BSON, un format directement mappé aux dictionnaires Python. Cela rend la conversion intuitive et naturelle.

#### Pourquoi ne pas insérer directement le DataFrame ?

MongoDB ne comprend pas directement les DataFrames pandas. Voici les problèmes principaux :

    Structure incompatible : Un DataFrame est une structure tabulaire alors que MongoDB utilise des documents JSON.
    Types de données : MongoDB utilise des types BSON spécifiques qui ne correspondent pas toujours directement aux types pandas.

In [1]:
import pandas as pd
import pymongo

In [55]:
client = pymongo.MongoClient()
database = client['exercices']
collection = database['kickstarter']

In [4]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head(10)

/var/folders/jq/5h45x39s35g2pjt_r9rnmwzr0000gn/T/ipykernel_7021/182781903.py:1: DtypeWarning: Columns (6,8,10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000.0,2011-08-17 06:31:31,1145.0,canceled,24,US,1145.0,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000.0,2012-07-23 20:46:48,1851.0,failed,28,US,1851.0,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000.0,2016-07-01 21:55:54,7534.0,successful,254,US,3796.0,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000.0,2017-08-28 15:47:02,16298.0,successful,367,US,2670.0,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000.0,2016-01-25 17:37:10,1.0,failed,1,CA,0.708148,0.738225
5,1771789139,Room For Growth!,Couture,Fashion,USD,2016-05-02,2000.0,2016-04-11 18:15:00,6.0,failed,2,US,6.0,6.000000
6,1301627822,Build a mini Udemy with Laravel and Vuejs,Web,Technology,CAD,2018-01-11,3000.0,2017-12-12 01:37:26,155.0,live,7,CA,15.563996,123.811806
7,881336601,Applitizer,Software,Technology,HKD,2017-08-20,100000.0,2017-07-21 16:50:05,1000.0,failed,1,HK,128.08538,127.831467
8,1669374237,"Help I Am Wolves fund their first album ""ABCD""...",Music,Music,EUR,2017-12-04,1000.0,2017-10-27 16:55:47,1465.0,successful,46,BE,0.0,1720.210418
9,556821416,Eco Bamboo Underwear funding....GOT WOOD ?,Fashion,Fashion,GBP,2012-12-13,9500.0,2012-11-13 10:58:34,1212.0,failed,43,GB,1928.137119,1953.672808


Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [57]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

## Question 0

### Netoyer les données

In [58]:
df_ks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   ID                150000 non-null  int64  
 1   name              149998 non-null  object 
 2   category          150000 non-null  object 
 3   main_category     150000 non-null  object 
 4   currency          150000 non-null  object 
 5   deadline          150000 non-null  object 
 6   goal              150000 non-null  object 
 7   launched          150000 non-null  object 
 8   pledged           150000 non-null  object 
 9   state             150000 non-null  object 
 10  backers           150000 non-null  object 
 11  country           150000 non-null  object 
 12  usd pledged       148518 non-null  object 
 13  usd_pledged_real  150000 non-null  float64
dtypes: float64(1), int64(1), object(12)
memory usage: 16.0+ MB


In [6]:
# Clean data 

if df_ks.isna().any().any(): 
    print("Il y a des valeurs manquantes dans le tableau.")
else:
    print("Il n'y a pas de valeurs manquantes dans le tableau.")
    
df_ks_cleaned = df_ks.dropna()
print(f"Nombre de lignes avant nettoyage : {len(df_ks)}")
print(f"Nombre de lignes après nettoyage : {len(df_ks_cleaned)}")
print('\n')

df_ks_cleaned["name"] = df_ks_cleaned["name"].astype(str)
df_ks_cleaned["category"] = df_ks_cleaned["category"].astype(str)
df_ks_cleaned["main_category"] = df_ks_cleaned["main_category"].astype(str)
df_ks_cleaned["currency"] = df_ks_cleaned["currency"].astype(str)

#df_ks_cleaned.iloc[65465]
#df_ks_cleaned.iloc[65464]
invalid_indexes = df_ks_cleaned[df_ks_cleaned["deadline"] == "USD"].index
print("Lignes problématiques avant suppression :")
print(df_ks_cleaned.loc[invalid_indexes])
df_ks_cleaned = df_ks_cleaned.drop(index=invalid_indexes)

df_ks_cleaned["deadline"] = pd.to_datetime(df_ks_cleaned["deadline"])
df_ks_cleaned["goal"] = df_ks_cleaned["goal"].astype(float)
df_ks_cleaned["launched"] = pd.to_datetime(df_ks_cleaned["launched"])
df_ks_cleaned["pledged"] = df_ks_cleaned["pledged"].astype(float)
df_ks_cleaned["state"] = df_ks_cleaned["state"].astype(str)
df_ks_cleaned["backers"] = df_ks_cleaned["backers"].astype(int)
df_ks_cleaned["country"] = df_ks_cleaned["country"].astype(str)
df_ks_cleaned["usd pledged"] = df_ks_cleaned["usd pledged"].astype(float)

df_ks_cleaned = df_ks_cleaned.rename(columns={"usd pledged": "usd_pledged"})

print(df_ks_cleaned["name"].dtypes)
print(df_ks_cleaned["category"].dtypes)
print(df_ks_cleaned["main_category"].dtypes)
print(df_ks_cleaned["currency"].dtypes)
print(df_ks_cleaned["deadline"].dtypes)
print(df_ks_cleaned["goal"].dtypes)
print(df_ks_cleaned["launched"].dtypes)
print(df_ks_cleaned["pledged"].dtypes)
print(df_ks_cleaned["state"].dtypes)
print(df_ks_cleaned["backers"].dtypes)
print(df_ks_cleaned["country"].dtypes)
print(df_ks_cleaned["usd_pledged"].dtypes)
print(df_ks_cleaned["usd_pledged_real"].dtypes)

Il y a des valeurs manquantes dans le tableau.
Nombre de lignes avant nettoyage : 150000
Nombre de lignes après nettoyage : 148516


Lignes problématiques avant suppression :
             ID                            name          category  \
66141  85964225  Debut Album from Michael Adam   Grace is Leaving   

      main_category currency deadline        goal launched  \
66141    Indie Rock    Music      USD  2014-04-17    700.0   

                   pledged  state     backers country usd pledged  \
66141  2014-04-02 21:56:35  850.0  successful      32          US   

       usd_pledged_real  
66141             850.0  
object
object
object
object
datetime64[ns]
float64
datetime64[ns]
float64
object
int64
object
float64
float64


/var/folders/jq/5h45x39s35g2pjt_r9rnmwzr0000gn/T/ipykernel_7021/349359228.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ks_cleaned["name"] = df_ks_cleaned["name"].astype(str)
/var/folders/jq/5h45x39s35g2pjt_r9rnmwzr0000gn/T/ipykernel_7021/349359228.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ks_cleaned["category"] = df_ks_cleaned["category"].astype(str)
/var/folders/jq/5h45x39s35g2pjt_r9rnmwzr0000gn/T/ipykernel_7021/349359228.py:15: SettingWithCopyWarning: 
A value is trying to be set 

In [69]:
df_ks_cleaned.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd_pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000.0,2011-08-17 06:31:31,1145.0,canceled,24,US,1145.000000,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000.0,2012-07-23 20:46:48,1851.0,failed,28,US,1851.000000,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000.0,2016-07-01 21:55:54,7534.0,successful,254,US,3796.000000,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000.0,2017-08-28 15:47:02,16298.0,successful,367,US,2670.000000,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000.0,2016-01-25 17:37:10,1.0,failed,1,CA,0.708148,0.738225


### Importer les données

In [72]:
client.drop_database('exercices')

In [7]:
client = pymongo.MongoClient()
database = client['exercices']
collection = database['kickstarter']

# Conversion DataFrame -> Liste de dictionnaires
data = df_ks_cleaned.to_dict(orient="records")

# Insertion des données
collection.insert_many(data)

print("Données insérées avec succès !")

ServerSelectionTimeoutError: localhost:27017: [Errno 61] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 675ebfd29329040f0d8fc394, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 61] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>

In [76]:
print(client.list_database_names())
print(database.list_collection_names())

['admin', 'config', 'exercices', 'local', 'series']
['kickstarter']


## Question 1  

In [77]:
# Récupérer les 5 projets ayant reçu le plus de promesse de dons.

top_pledged_projects = collection.find().sort('pledged', pymongo.DESCENDING).limit(5)

for project in top_pledged_projects:
    print(project['name'], project['pledged'])

COOLEST COOLER: 21st Century Cooler that's Actually Cooler 13285226.36
Pebble 2, Time 2 + All-New Pebble Core 12779843.49
Expect the Unexpected. digiFilmï¿½ Camera by YASHICA 10035296.0
OUYA: A New Kind of Video Game Console 8596474.58
The Everyday Backpack, Tote, and Sling 6565782.5


## Question 2

In [78]:
# Compter le nombre de projets ayant atteint leur but.

successful_projects_count = collection.count_documents({'state': 'successful'})
print(f"Nombre de projets ayant atteint leur objectif : {successful_projects_count}")

Nombre de projets ayant atteint leur objectif : 52998


## Question 3

In [79]:
# Compter le nombre de projets pour chaque catégorie.

pipeline = [
    {
        '$group': {
            '_id': '$category',  # Regrouper par la catégorie
            'count': {'$sum': 1}  # Compter le nombre de projets dans chaque catégorie
        }
    },
    {
        '$sort': {'count': -1}  # Trier par nombre de projets (descendant)
    }
]

# Exécuter l'agrégation
category_counts = collection.aggregate(pipeline)

# Afficher les résultats
for category in category_counts:
    print(f"Catégorie: {category['_id']}, Nombre de projets: {category['count']}")

Catégorie: Product Design, Nombre de projets: 8885
Catégorie: Documentary, Nombre de projets: 6497
Catégorie: Tabletop Games, Nombre de projets: 5579
Catégorie: Music, Nombre de projets: 5293
Catégorie: Shorts, Nombre de projets: 4857
Catégorie: Video Games, Nombre de projets: 4797
Catégorie: Food, Nombre de projets: 4612
Catégorie: Fiction, Nombre de projets: 3703
Catégorie: Film & Video, Nombre de projets: 3657
Catégorie: Nonfiction, Nombre de projets: 3390
Catégorie: Fashion, Nombre de projets: 3379
Catégorie: Art, Nombre de projets: 3358
Catégorie: Apparel, Nombre de projets: 2827
Catégorie: Theater, Nombre de projets: 2786
Catégorie: Rock, Nombre de projets: 2707
Catégorie: Technology, Nombre de projets: 2689
Catégorie: Children's Books, Nombre de projets: 2686
Catégorie: Apps, Nombre de projets: 2535
Catégorie: Webseries, Nombre de projets: 2316
Catégorie: Photography, Nombre de projets: 2239
Catégorie: Indie Rock, Nombre de projets: 2192
Catégorie: Publishing, Nombre de projets:

## Question 4

In [ ]:
# Compter le nombre de projets français ayant été instanciés avant 2016.

from datetime import datetime

date_limit = datetime(2016, 1, 1)

count = collection.count_documents({
    'country': 'FR',               
    'launched': {'$lt': date_limit}         # Opérateur MongoDB 'less than' : val
})
print(f"Nombre de projets français lancés avant 2016 : {count}")

Nombre de projets français lancés avant 2016 : 330


## Question 5

In [84]:
# Récupérer les projets américains ayant demandé plus de 200 000 dollars.

projects = collection.find({
    'country': 'US',             
    'goal': {'$gt': 200000}         # Opérateur MongoDB 'greater than' : val      
})

for project in projects:
    print(project)

{'_id': ObjectId('6758828ca03ea151f32eabba'), 'ID': 866634482, 'name': 'A CALL TO ADVENTURE', 'category': 'Film & Video', 'main_category': 'Film & Video', 'currency': 'USD', 'deadline': datetime.datetime(2012, 9, 14, 0, 0), 'goal': 287000.0, 'launched': datetime.datetime(2012, 8, 13, 23, 14, 2), 'pledged': 1465.0, 'state': 'failed', 'backers': 11, 'country': 'US', 'usd_pledged': 1465.0, 'usd_pledged_real': 1465.0}
{'_id': ObjectId('6758828ca03ea151f32eac6c'), 'ID': 993194166, 'name': 'Storybricks, the storytelling online RPG', 'category': 'Video Games', 'main_category': 'Games', 'currency': 'USD', 'deadline': datetime.datetime(2012, 6, 1, 0, 0), 'goal': 250000.0, 'launched': datetime.datetime(2012, 5, 1, 20, 49, 58), 'pledged': 23680.54, 'state': 'failed', 'backers': 409, 'country': 'US', 'usd_pledged': 23680.54, 'usd_pledged_real': 23680.54}
{'_id': ObjectId('6758828ca03ea151f32eac74'), 'ID': 1147175344, 'name': 'Shine On New World', 'category': 'Theater', 'main_category': 'Theater', 

## Question 6 

In [85]:
# Compter le nombre de projet ayant "Sport" dans leur nom

count_sport_projects = collection.count_documents({
    'name': {'$regex': 'Sport', '$options': 'i'}  # Recherche insensible à la casse
})

print(f"Nombre de projets ayant 'Sport' dans leur nom : {count_sport_projects}")

Nombre de projets ayant 'Sport' dans leur nom : 500
